# Homework 1: Feed-forward Neural Networks (100 points)

### Overview

Below you will find a PyTorch implementation of a feed-forward neural network for image recognition. We use the popular MNIST dataset, where the model predicts a single digit (0-9) for a black-and-white photo of a handwritten digit. This is a _classification_ task.

### NN Architecture

Each image has size 28x28 grayscale pixel values between 0 and 255. In preprocessing, we flatten each image to a single vector of length $28^2 = 784$, which serves as the entire input for the model.

For each image, we aim to predict one of ten classes (0-9). We could use an output layer $y$ of size 1 (a single neuron) -- for example, using a naive mapping like prediction $p = \mathrm{int}(10y)$. But this presupposes that a handwritten 0 is similar to a handwritten 1 and very different than a handwritten 9, which isn't the case. So instead we use an output layer $y$ of size 10, where the prediction $p = argmax(y)$, so each output neuron controls the likelihood for a particular class.

We use a simple two-layer neural network. To begin, we will have an input size of 784, a hidden layer of size 5, and an output layer of size 10.

### Your Task

At the bottom of this notebook file, there are a series of questions testing your understanding of this neural network architecture. Some questions include instructions where you will need to modify hyperparameters (notated in the code below) and re-run the model to investigate the changed results. __There is no need to read through the following code in depth to answer the questions, but it may be useful as a reference.__

Below each question is a cell with the text “Type Markdown and LaTex.” Double-click the cell and type your response to the question. Save your responses by clicking on the floppy disk icon or choosing File - Save and Checkpoint.

After responding to the questions, download your notebook as a .PDF file by choosing File - Download as - pdf (.pdf). You will be submitting this .pdf to your instructor for grading.

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
torch.manual_seed(0)

In [2]:
root_dir = 'assets_week1'
trainDataset = datasets.MNIST(root=root_dir, train=True, transform=transforms.ToTensor(), download=True)
testDataset = datasets.MNIST(root=root_dir, train=False, transform=transforms.ToTensor())

In [3]:
class NNModel(nn.Module):
    def __init__(self, inputSize, outputSize, hiddenSize, activate):
        super().__init__()
        
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.Tanh() if activate == "Tanh" else nn.ReLU()
        self.layer1 = nn.Linear(inputSize, hiddenSize)
        self.layer2 = nn.Linear(hiddenSize, outputSize)
        
    def forward(self, X):
        hidden = self.activate(self.layer1(X))
        return self.layer2(hidden)

In [4]:
# The dimensionality of the input
inputSize = 784
# Number of neurons in the first layer
hiddenSize = 300
# Number of neurons in the second layer
outputSize = 10
# Activation function (default: ReLU, options: Sigmoid, Tanh, ReLU)
activation = "ReLU"
# Learning rate
learningRate = 0.001
# Number of training epochs
numEpochs = 5
# Number of training examples per batch
batchSize = 200

In [5]:
trainLoader = torch.utils.data.DataLoader(dataset=trainDataset, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(dataset=testDataset, batch_size=batchSize, shuffle=False)

net = NNModel(inputSize, outputSize, hiddenSize, activation)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learningRate)

print('>>> Beginning training!')
for epoch in range(numEpochs):
    for i, (images, labels) in enumerate(trainLoader):
        images = images.view(-1, 28*28)
        
        optimizer.zero_grad()
        
        # Forward propagation
        outputs = net(images)
        
        # Backpropagation
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient descent
        optimizer.step()
        
        # Logging
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {}'.format(epoch+1, numEpochs, i+1, len(trainDataset)//batchSize, loss))

print()
print('>>> Beginning validation!')
correct, total = 0, 0
for i, (images, labels) in enumerate(testLoader):
    images = images.view(-1, 28*28)
    
    outputs = net(images)
    _, prediction = torch.max(outputs, axis=1)
    correct += torch.sum(prediction == labels)
    total += labels.size(0)
print('Validation accuracy: {}%'.format(correct.item()/total*100))

>>> Beginning training!
Epoch [1/5], Step [100/300], Loss: 0.3419055938720703
Epoch [1/5], Step [200/300], Loss: 0.3129781484603882
Epoch [1/5], Step [300/300], Loss: 0.2323944866657257
Epoch [2/5], Step [100/300], Loss: 0.17499448359012604
Epoch [2/5], Step [200/300], Loss: 0.15201827883720398
Epoch [2/5], Step [300/300], Loss: 0.11334750056266785
Epoch [3/5], Step [100/300], Loss: 0.13201119005680084
Epoch [3/5], Step [200/300], Loss: 0.07654597610235214
Epoch [3/5], Step [300/300], Loss: 0.11779850721359253
Epoch [4/5], Step [100/300], Loss: 0.0658668801188469
Epoch [4/5], Step [200/300], Loss: 0.0969599187374115
Epoch [4/5], Step [300/300], Loss: 0.0748835951089859
Epoch [5/5], Step [100/300], Loss: 0.0826909989118576
Epoch [5/5], Step [200/300], Loss: 0.032050978392362595
Epoch [5/5], Step [300/300], Loss: 0.053437910974025726

>>> Beginning validation!
Validation accuracy: 97.45%


## Homework Questions

Your goal is to improve the model's accuracy by tuning hyperparameters. If a question asks you to modify a hyperparameter and you obtain improved results, retain that hyperparameter change for subsequent questions. Otherwise, revert back to the original hyperparameter value.

### Question 1: Loss Minimization & Gradient Descent (5 points)

Given a neural network with model parameters $\theta$, loss function $E$, and learning rate $\alpha$, what is the correct method to perform gradient descent?

a) $\theta_i += \alpha E$

b) $\theta_i -= \alpha E$

c) $\theta_i += \alpha\frac{\partial E}{\partial \theta_i}$

d) $\theta_i -= \alpha\frac{\partial E}{\partial \theta_i}$

Answer: d) $\theta_i -= \alpha\frac{\partial E}{\partial \theta_i}$. In order to estimate the model parameter 𝜃 , we must take the partial derivative of the loss function with respect to 𝜃, and multiply it by the learning rate. We also want to move in a "negative" direction when performing gradient descent, not a positive direction, in order to find the model parameters that minimize the loss function.

### Question 2: Class Imbalance (10 points)

Imagine you are an engineer tasked with helping a company to identify faulty parts early using an machine learning-based image recognition system. What evaluation metric would you use? More specifically, explain why a raw percent accuracy score would be a poor choice of evaluation metric for this problem space.

Raw percent accuracy is often not enough information to make the decision regarding whether or not a model is good enough for solving a particular problem. This problem in particular seems to be a binary classification problem - in other words, the part is designated as faulty vs not faulty. Assuming the machinery manufacturer makes functional parts the vast majority of the time, then there would likely be a large class imbalance in this given situation. The majority of the time, the parts input into the image recognition model would be fully functional, so if the machine learning engineer chooses to maximize model accuracy as their primary metric, the model would simply be able to always predict the majority class - not faulty - and still achieve a very high accuracy. This, of course, defeats the entire purpose of the model in the first place, which is to discover faulty parts.


In this case, a better evaluation metric might be cost-minimization, by creating a more cost-sensitive neural network model. To do this, the engineer can place a higher error weighting on the important class - faulty - and a much smaller weight on the less imporant class, which is functional. In other words, large weight for the minority class, and small weight for the majority class. The engineer can modify their evaluation criteria to focus on minimizing the error penalty on the minority class as much as possible.

### Question 3a:  Size of a Hidden Layer (10 points)

Explain how the hidden layer size influences the architecture of a feed-forward neural network. In doing so, note what can happen if the hidden size is too large and what can happen if the hidden size is too small.

It is important to consider whether having multiple layers in one's machine learning model might be a good choice for the particular problem at hand. In general, using a single layer would only be wise if the model has classes that are linearly seperable. If the problem is more complicated, using multiple layers may be a better choice. 

As for the number of nodes in each hidden layer(size), it is better to utilize intuition and experimentation to avoid choosing too many nodes or too little nodes. Too many nodes(too large size) can result in the model overfitting to the training data. In this case, the model has so much processing capacity that, if the amount of the training data is insufficient, there will not be enough information in the training set to train all of the neurons in the layer. Furthermore, it can also increase the time needed to adequately train the network and render it unusable. A too small hidden layer size, on the other hand, can cause the the model to underfit and not adequately detect signals in the data set.



### Question 3b: Size of a Hidden Layer  (10 points)

Increase the hidden size from 5 to 300 and re-run your trial. How does the accuracy change?

_a) It increases, since the model learns more quickly_

_b) It increases, since the model has more memory and can learn more complex features_

_c) It decreases, since the model has to learn more parameters and it doesn't have enough time_

_d) It decreases, since the model has less memory_

Answer: _b) It increases, since the model has more memory and can learn more complex features_. A hidden layer with too low dimensionality may insufficiently capture important signal in the model, while a hidden layer with a too high dimensionality will cause the model instead simply memorize the training data as opposed to fitting the general shape. It seems as if increasing the hidden layer size to 300 allows us to fit the test data very well, but it would be important to run some more tests on even higher hidden layer sizes to see when the detrimental effects of overfitting start to become noticeable in the accuracy score.

### Question 4a: Learning Rate  (10 points)

Explain the purpose of a learning rate. In doing so, note what can happen if the learning rate is too large and what can happen if the learning rate is too small.

The learning rate is the neural network hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated during the optimization process, using stochastic gradient descent. When updating the model at each state, the stochastic gradient descent algorithm estimates the error for the model at that state and then updates the model weights accordingly using back-propogation. The amount that the weights are updated during this process is called the learning rate(the parameter can be set manually, and typically ranges between 0-1). If the learning rate is too small, it is possible that the model may take an inordinately long time to train and get "stuck", while a learning rate that is too large may cause the model to converge to a suboptimal solution too quickly.



### Question 4b: Learning Rate  (10 points)

Increase the learning rate from 0.001 to 1. How does the accuracy change?

a) It increases, since the model learns more quickly

b) It increases, since the model is better able to converge

c) It decreases, since the model learns too slowly

d) It decreases, since the model is not able to converge

Answer: d) It decreases, since the model is not able to converge. Because we made the learning rate substantially higher, the model converges on a solution too quickly. Because it converges on a solution more quickly at higher learning rates, it becomes increasingly likely as the learning rate increases that the solution the model ultimately converges on is not the most "globally" optimal one.

### Question 5a: Activation Functions (10 points)

Explain the two main purposes of activation functions in feed-forward neural networks. Also, explain the main benefit of the Tanh activation function over the Sigmoid activation function, and the main benefit of the ReLU activation function over the Sigmoid activation function.



There are two important purposes that the choice of the activation function will have in the development of a feed-forward neural network. One would be the performance of the model, in the sense that some activation functions are more computationally intensive than others. The Relu activation function, for instance, is simple and also computationally efficient, as it simply replaces all negative input values with 0. A second very important purpose of activation functions is the ability for them to add non-linearity into a neural network. While there are some activation functions that produce linear decision boundaries between classes, the vast majority of real world problems are not linear.

Other important features that activation functions should have are the ability to be differentiable, since this is necessary for stochastic gradient descent to work properly, and should also not cause massive changes in the distribution of the inputs(covariate shift). Additionally, it would be ideal for the activation function to not be easily susceptible to the vanishing gradient problem. This problem occurs when the activation function causes the gradient to decrease exponentially towards zero duruing back propogation for each training session. This implies that the initial layers will not have their weights and biases updated effectively, and since the initial layers are important for recognizing the input data, it can result in innacurracies all throughout the neural net.

The Tanh activation function is beneficial to use over the sigmoid activation function because, while it is also sigmoidal in shape, the range of the tanh function is greater (inputs are mapped between 0,1 in the former and -1,1 in the latter). This allows negative inputs to be mapped to strongly negative values and near zero inputs to be mapped to near zero values. Tanh also has a stronger gradient than sigmoid(steeper derivatives).

The ReLU function is beneficial to use over the sigmoid activation function because the activation is much less dense than the sigmoid function. By utilizing the sigmoid function, essentially every neuron in the network is fired up, which can become computationally slow and costly when there are multiple millions of neurons and more in the network. The ReLU function makes the activation sparse by not activating a large percentage of neurons(output zero for negative inputs).

### Question 5b: Activation Functions (5 points)

Change the activation function in the hyperparameter list above to determine which activation function is most effective at this task.

a) ReLU: 97.45%

b) Sigmoid: 95.49%

c) Tanh: 96.93%

Although the accuracy differences between the functions are marginal, it seems as if ReLU provides the best accuracy for this given classification problem.

a) ReLU: 97.45%

### Question 6: Overfitting  (10 points)

Define overfitting and explain how it can damage model training and results.

The objective when developing any neural network model is to ensure that it performs well on both the data used to train it as well as the unseen data that is held out for testing. A model that can do that is said to be good at generalizing. If the model learns too little when training on the training dataset, it will likely perform poorly on the training data and even worse on the test data, which is a sign of underfitting. However, if the model learns too much from the training data, it will be bad at generalizing, and, though it may perform very well on the training data, it will likely perform poorly on the test data. This is a salient indicator of overfitting.

Model fit can also be looked at in terms of the bias-variance tradeoff. If the model is underfit, then it will essentially have high bias and low variance, meaning it will always make assumptions that the target function/result will be a certain way and thus will be unable to learn the problem regardless of the examples presented in the training data. In essence, it pays very little attention to the training data. An overfit model, on the other hand, has low bias but high variance, meaning that it pays quite a bit of attention to the training data and, instead of oversimplifying the model as in underfitting, it instead makes the model too complex, which prevents it from fitting well to held out data where the underlying distrubution may be slightly different. 





### Question 7: Early Stopping  (10 points)

Outline a procedure for early stopping to prevent overfitting.

Early stopping essentially just implies that one stops the neural network training procedure before it converges on a solution, with the intention that, by doing so, one is able to stop the model from getting to a stage where it is overfit and thus, reduces the error incurred on the testing dataset. The state where it is stopped would ideally be in an optimal "in between" spot, where it is not highly underfit nor highly overfit.

A hyperparamter that can be used to govern the stopping point is the 'epoch' parameter, which defines how many times the full training dataset is iterated through during training. By increasing it or decreasing it greatly, one can easily cause the model to overfit or underfit. A potential way to find a good early stopping point for the model would be to continue incrementing the epoch parameter and monitor the model accuracy score on the testing set until the accuracy score starts to experience diminishing returns. 

### Question 8: Regularization  (10 points)

Briefly explain a few common methods of regularization to prevent overfitting.

Dropout: With the dropout regularization method, during training, a certain percentage of neurons in the network are not activated by the activation function. In other words, they will simply output zero. Typically, between 15-20% of the nodes are dropped. By doing this, the neural network will become more robust at better at generalizing because it will not solely rely on any one connection in the network when making predictions.

Data Augmentation: Data augmentation essentially means increasing the diversity of the training data through the additions of manipulated fake input data, while preserving that data's output class label, which is done to make the model more robust to varied inputs. Some of these augmentation techniques include rotation, cropping, mirroring, and jittering. It is worth noting that this regularization technique is more particularly suited for specific data domains, such as with image data.

Weight Regularization: This is perhaps the simplest and most common method of regularization. Weight regularization implies that a penalty is added to the loss function during each training iteration corresponding to the size of the model weights. This encourages the model to map the inputs to the outputs in such a way that the weights are kept small. Another flavor similar to this is activation regularization, where the model is penalized based on the magnitude of the activations. 